https://stackoverflow.com/questions/54051495/cross-operations-between-two-datafarames

In [1]:
import numpy as np
import pandas as pd
df1 = pd.DataFrame(10*(2+np.random.randn(500, 3)), columns=list('ABC'))
df2 = pd.DataFrame(10*(2+np.random.randn(500, 3)), columns=list('CDA'))
df1.drop(df1.index[[1, 4, 284, 354, 498]], inplace=True)
df2.drop(df2.index[[0, 98, 159]], inplace=True)
df1.loc[2, 'B'] = np.nan
df1.loc[5, 'C'] = np.nan
df1.loc[3, 'A'] = np.nan
df2.loc[5, 'C'] = np.nan
df2.loc[1, 'D'] = np.nan
df2.loc[2, 'A'] = np.nan

In [3]:
df1.head(10)

,A,B,C
0,20.438695,18.114421,20.445370
2,12.789906,NaN,3.988319
3,NaN,11.026463,9.421921
5,19.919580,7.462012,NaN
6,21.290647,23.952295,-10.354758
7,8.447708,14.710224,25.499204
8,16.603850,24.862611,20.354342
9,6.088232,15.066117,22.906491
10,19.621493,15.877428,15.149765
11,2.052592,9.031476,19.531663


In [4]:
df2.head(10)

,C,D,A
1,15.159127,NaN,29.432163
2,23.449304,8.393440,NaN
3,6.057011,32.881258,21.033391
4,31.162671,4.128745,23.264304
5,NaN,32.796018,11.171984
6,32.019817,14.603303,33.106655
7,17.566806,26.804403,12.421421
8,30.121336,46.520462,41.934098
9,13.498463,30.170049,24.221281
10,19.554489,28.238385,26.284620


In [6]:
# Merge the dataframes
merged_df = df1.merge(df2, right_index = True, left_index = True, how = 'outer')
# Create dataframes for each formula
simple_ret_df = pd.DataFrame(index = merged_df.index) # for (1 - df1 / df2)
log_ret_df    = pd.DataFrame(index = merged_df.index) # for ln(df1 / df2)
for column in merged_df.columns:
    # Get initial letter from the column names
    i = column[0]
    # filter only the columns that start with the same letter
    df_test = merged_df[merged_df.columns[merged_df.columns.str.startswith(i)]]
    #If only one column, add that to the dataframes
    if df_test.shape[1] == 1:
        simple_ret_df = simple_ret_df.merge(df_test, right_index = True, left_index = True, how = 'outer')
        
        log_ret_df = log_ret_df.merge(df_test, right_index = True, left_index = True, how = 'outer')
    #If two columns do the calculations
    else:
        simple_ret_df.loc[:,i] = np.where((df_test[df_test.columns[0]].isin([np.nan, 0]))|(df_test[df_test.columns[1]].isin([np.nan, 0])), 
        np.nan,  (1- df_test[df_test.columns[0]]) / df_test[df_test.columns[1]])
        
        log_ret_df.loc[:,i] = np.where((df_test[df_test.columns[0]].isin([np.nan, 0]))|(df_test[df_test.columns[1]].isin([np.nan, 0])), 
        np.nan,  np.log(df_test[df_test.columns[0]] / df_test[df_test.columns[1]]))

#Adjust the names of the columns
simple_ret_df.columns = simple_ret_df.columns.str[0]
log_ret_df.columns    = log_ret_df.columns.str[0]

print('simple_ret_df.head(6)', '\n\n',simple_ret_df.head(6), '\n\n', 'log_ret_df.head(6)', '\n', log_ret_df.head(6))

simple_ret_df.head(6) 

           A          B         C          D
0       NaN  18.114421       NaN        NaN
1       NaN        NaN       NaN        NaN
2       NaN        NaN -0.127437   8.393440
3       NaN  11.026463 -1.390442  32.881258
4       NaN        NaN       NaN   4.128745
5 -1.693484   7.462012       NaN  32.796018 

 log_ret_df.head(6) 
           A          B         C          D
0       NaN  18.114421       NaN        NaN
1       NaN        NaN       NaN        NaN
2       NaN        NaN -1.771471   8.393440
3       NaN  11.026463  0.441823  32.881258
4       NaN        NaN       NaN   4.128745
5  0.578294   7.462012       NaN  32.796018


/Users/jpinzon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in log
